In [1]:
# Basics -----------------------------------------------------
import pickle
from collections import defaultdict # mayby use..
import time

import numpy as np
import pandas as pd
from scipy import sparse
from scipy import spatial


import matplotlib.pyplot as plt
import seaborn as sns
import regex
import re


# NLTK -----------------------------------------------------

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
#stop_words = set(nltk.corpus.stopwords.words("english"))

# Word2vec ----------------------------------------------------
import gensim

# SK learn --------------------------------------------------

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import TruncatedSVD

from sklearn import metrics

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Sampling -------------------------------------------------

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import RandomUnderSampler



# data imports

In [2]:
# DATA ------------------------------------------------------
df = pd.read_csv("labled_tweets.csv", index_col=0)

#df_unlabled_big = pd.read_csv("/media/polichinel/DATA/backup/PDS_DATA/new_many_tweets.csv", index_col= 0)
df_unlabled_big1 = pd.read_csv("new_all_unlabeled_tweets1.csv", index_col= 0)
df_unlabled_big2 = pd.read_csv("new_all_unlabeled_tweets2.csv", index_col= 0)

df_unlabled = pd.read_csv("all_unlabled_tweets.csv", index_col= 0)
df_labled = pd.read_csv("labled_tweets.csv", index_col= 0)

pkl_file = open('full_remove.pkl', 'rb') # from feature set.
full_remove = pickle.load(pkl_file)
pkl_file.close()

/home/polichinel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,1,4,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# check it is the right full remove:
full_remove
# Should be
# ['@', 'a', '...', 'rt', ':', ',', 'to', 'the', '#']

['@', 'a', '...', 'rt', ':', ',', 'to', 'the', '#']

In [4]:
path = "/media/polichinel/DATA/backup/PDS_DATA/"

spec_path = path + "hot_comments_politics.pkl"


pkl_file = open(spec_path, 'rb') # from feature set.
comments1 = pickle.load(pkl_file)
pkl_file.close()

# ---------------------------------------

spec_path = path + "controversial_comments_politics.pkl"

pkl_file = open(spec_path, 'rb') # from feature set.
comments2 = pickle.load(pkl_file)
pkl_file.close()

# ---------------------------------------

spec_path = path + "hot_comments_news.pkl"

pkl_file = open(spec_path, 'rb') # from feature set.
comments3 = pickle.load(pkl_file)
pkl_file.close()

# ---------------------------------------

spec_path = path + "controversial_comments_news.pkl"

pkl_file = open(spec_path, 'rb') # from feature set.
comments4 = pickle.load(pkl_file)
pkl_file.close()

# ---------------------------------------

spec_path = path + "top_comments_politics.pkl"


pkl_file = open(spec_path, 'rb') # from feature set.
comments5 = pickle.load(pkl_file)
pkl_file.close()


# ---------------------------------------

spec_path = path + "top_comments_news.pkl"


pkl_file = open(spec_path, 'rb') # from feature set.
comments6 = pickle.load(pkl_file)
pkl_file.close()

# ---------------------------------------

comments_all = comments1 + comments2 + comments3 + comments4 + comments5 + comments6
all_text = comments_all + list(df_unlabled_big1.text) + list(df_unlabled_big2.text) + list(df_unlabled.text)

# ---------------------------------------

len(all_text)


2539464

# Spilt:

In [5]:
y = df["lacking_dem_pref"]
X = df["text"]

# Outer layer
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(X, y, test_size=0.20, random_state=42)

# inner layer
X_train, X_test, y_train, y_test = train_test_split(X_train_out, y_train_out, test_size=0.20, random_state=42)


# Function you migth use - remember to sort them..

In [6]:
def roc_test(X, save_name = "roc_test.pdf", title = "some_title", model_names = ""):
    """X should be a list of tuples, where each tuple contains X_train at [0] and X_test at [1]"""
    plt.figure(figsize=[8,8])
    count = 0
    
    for i in X:
        clf_lg = LogisticRegression().fit(i[0], y_train)
        pred_proba_log = clf_lg.predict_proba(i[1])[:,1]
        fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_proba_log)
        auc = round(metrics.roc_auc_score(y_test, pred_proba_log), 3)
        plt.plot(fpr, tpr, label = model_names[count] + "AUC = " + str(auc))
        count +=1

    plt.plot(np.arange(0, 1.1, 0.1), np.arange(0, 1.1, 0.1), linestyle = "dashed", color = "orange")

    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.title(title, fontsize = 20)
    plt.xlabel('False Positive Rate', fontsize = 18)
    plt.ylabel('True Positive Rate', fontsize = 18)
    plt.grid(True)

    plt.legend(fontsize = 16)
    sns.despine(left=True, bottom= True)

    plt.savefig(save_name, bbox_inches="tight")
    plt.show()

In [7]:
stop_words = set (nltk.corpus.stopwords.words("english"))

def prep2(text):
    wordlist = nltk.word_tokenize(text)
    wordlist = [lemmatizer.lemmatize(w.lower()) for w in wordlist]
    wordlist = [w for w in wordlist if w not in stop_words and w not in full_remove]# foun in the outputs above
    return wordlist

In [8]:
# stop_words = []#["goptaxscam"] # for now no stopwords

# def prep2(text):
#     wordlist = nltk.word_tokenize(text)
#     wordlist = [lemmatizer.lemmatize(w.lower()) for w in wordlist]
#     wordlist = [w for w in wordlist if w not in full_remove and w not in stop_words]
#     return wordlist

# # but here yor are tokenizing for all wordS? Should it not only be train???
# # words_processed = prep2(df.text.to_string()) 
# # len(words_processed)

# preprocess

In [9]:
# tweets are back from 140 character limit; not 240!
%time sentences = [prep2(sent) for sent in nltk.sent_tokenize(pd.Series(all_text).to_string())]

CPU times: user 12min 27s, sys: 2.26 s, total: 12min 29s
Wall time: 12min 29s


# w2v

In [108]:
# Set values for various parameters
feature_size_grid = np.arange(200,500,50)    # Word vector dimensionality  
window_context_grid = [2,3,4,5,10]          # Context window size                                                                                    
min_word_count_grid = [20,30,40]   # Minimum word count             HERE           
sample_grid = [1e-3, 1e-4]   # Downsample setting for frequent words


In [109]:
all_models = defaultdict()
counter = 0

for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                #print(counter)
                counter += 1
                
total_number_of_models = counter
total_number_of_models

180

see https://radimrehurek.com/gensim/models/word2vec.html for why it is hard to make complete ly determenistic

In [110]:
#all_models = defaultdict()
counter = 1
hyper_parameters = defaultdict(list)


program_starts = time.time()
for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                                
                spec_model = gensim.models.Word2Vec(sentences, size=fs, window=wc, min_count=mwc, sample=s, sg = 1, seed = 42) # helps, but still not deterministic
                
                #------------------------------------------------------------------------------------------------------
                
                word2vec_model = spec_model.wv
    
                n_features = word2vec_model.word_vec("potus").shape[0]
                index2word_set = set(word2vec_model.index2word)

                def avg_feature_vector(sentence, model, num_features, index2word_set):
                    words = sentence.split()
                    feature_vec = np.zeros((num_features, ), dtype='float32')
                    n_words = 0
                    for word in words:
                        if word in index2word_set:
                            n_words += 1
                            feature_vec = np.add(feature_vec, model[word])
                    if (n_words > 0):
                        feature_vec = np.divide(feature_vec, n_words)
                    return(feature_vec)

                def word_vectorizer(X):
                    """Where X is either X_test or X_train"""
                    data_set = pd.DataFrame(np.zeros([len(X),n_features]))
                    #vectors = np.zeros(n_features)
                    #dividend = 0 # you can remove this one right?


                    for i in range(len(X)):
                        sent_list = []
                        strings = X.iloc[i].split()
                        for j in range(len(strings)):
                            word = prep2(strings[j].lower())
                            if len(word) > 0:
                                clean_word = re.sub(r'[^\w\s]','',word[0])
                                sent_list.append(clean_word)
                                sent = " ".join(sent_list)

                                data_set.iloc[i] = avg_feature_vector(sent, model=word2vec_model, num_features=n_features, index2word_set=index2word_set)

                    return(data_set)

                w2v_train_df_new = word_vectorizer(X_train)
                w2v_test_df_new = word_vectorizer(X_test)

                clf_lg = LogisticRegression().fit(w2v_train_df_new, y_train)
                pred_proba_log = clf_lg.predict_proba(w2v_test_df_new)[:,1]

                auc = round(metrics.roc_auc_score(y_test, pred_proba_log), 3)

                
                
                #------------------------------------------------------------------------------------------------------

                
                
                model_name = "model_" + str(counter)
                hyper_parameters[model_name] = (auc,[fs, wc, mwc, s])
                                
                
                now = time.time()
                time_spend =  now - program_starts
                time_model = time_spend/counter
                total_time = (time_model) * total_number_of_models 
                
                print(str(counter)+"/"+str(total_number_of_models)+" done" + "\tAUC: {}\t hyper_par: {}, {}, {}, {}\t timer: {:.3} minutes".format(auc, fs, wc, mwc, s, time_spend/60))
                print("\tExpected time pr. model: {:.3} minutes".format(time_model/60))               
                print("\tExpected time total: {:.3} minutes".format(total_time/60))
                print("\tExpected time remaining: {:.3} minutes\n".format((total_time-time_spend)/60))

                counter += 1

1/180 done	AUC: 0.893	 hyper_par: 200, 2, 20, 0.001	 timer: 2.49 minutes
	Expected time pr. model: 2.49 minutes
	Expected time total: 4.49e+02 minutes
	Expected time remaining: 4.46e+02 minutes

2/180 done	AUC: 0.9	 hyper_par: 200, 2, 20, 0.0001	 timer: 4.55 minutes
	Expected time pr. model: 2.27 minutes
	Expected time total: 4.09e+02 minutes
	Expected time remaining: 4.05e+02 minutes

3/180 done	AUC: 0.892	 hyper_par: 200, 2, 30, 0.001	 timer: 7.04 minutes
	Expected time pr. model: 2.35 minutes
	Expected time total: 4.22e+02 minutes
	Expected time remaining: 4.15e+02 minutes

4/180 done	AUC: 0.896	 hyper_par: 200, 2, 30, 0.0001	 timer: 8.93 minutes
	Expected time pr. model: 2.23 minutes
	Expected time total: 4.02e+02 minutes
	Expected time remaining: 3.93e+02 minutes

5/180 done	AUC: 0.895	 hyper_par: 200, 2, 40, 0.001	 timer: 11.5 minutes
	Expected time pr. model: 2.29 minutes
	Expected time total: 4.13e+02 minutes
	Expected time remaining: 4.01e+02 minutes

6/180 done	AUC: 0.898	 hy

In [126]:
# Assesing the models:
good_list = []

for i, j in hyper_parameters.items():
    if j[0] >= 0.905:
        print(j)

(0.907, [200, 5, 20, 0.0001])
(0.908, [250, 10, 20, 0.001])
(0.905, [250, 10, 20, 0.0001])
(0.909, [300, 3, 30, 0.0001])
(0.912, [300, 10, 20, 0.001])
(0.905, [300, 10, 20, 0.0001])
(0.905, [350, 3, 20, 0.0001])
(0.905, [350, 5, 30, 0.0001])
(0.905, [350, 5, 40, 0.0001])
(0.907, [400, 3, 20, 0.0001])
(0.905, [400, 3, 30, 0.0001])
(0.906, [400, 4, 20, 0.0001])
(0.909, [400, 4, 30, 0.0001])
(0.906, [400, 10, 20, 0.001])
(0.909, [400, 10, 40, 0.001])
(0.905, [450, 2, 20, 0.0001])
(0.908, [450, 3, 20, 0.0001])
(0.906, [450, 4, 20, 0.0001])
(0.908, [450, 4, 30, 0.0001])
(0.905, [450, 5, 20, 0.0001])
(0.906, [450, 10, 20, 0.001])
(0.91, [450, 10, 30, 0.0001])


In [129]:
hyper_parameters1 = hyper_parameters.copy()

In [131]:
# A pickle for the word2vec implimentation
output = open('hyper_parameters1.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(hyper_parameters1, output)

output.close()

# Second run

In [137]:
# New elauated parameters
feature_size_grid = np.arange(300,600,50)    # Word vector dimensionality  
window_context_grid = [4,5,8,10,12]          # Context window size                                                                                    
min_word_count_grid = [15,20,25]   # Minimum word count             HERE           
sample_grid = [1e-4, 5e-5]   # Downsample setting for frequent words


In [138]:
all_models = defaultdict()
counter = 0

for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                #print(counter)
                counter += 1
                
total_number_of_models = counter
total_number_of_models

180

In [139]:
#all_models = defaultdict()
counter = 1
hyper_parameters = defaultdict(list)


program_starts = time.time()
for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                                
                spec_model = gensim.models.Word2Vec(sentences, size=fs, window=wc, min_count=mwc, sample=s, sg = 1, seed = 42) # helps, but still not deterministic
                
                #------------------------------------------------------------------------------------------------------
                
                word2vec_model = spec_model.wv
    
                n_features = word2vec_model.word_vec("potus").shape[0]
                index2word_set = set(word2vec_model.index2word)

                def avg_feature_vector(sentence, model, num_features, index2word_set):
                    words = sentence.split()
                    feature_vec = np.zeros((num_features, ), dtype='float32')
                    n_words = 0
                    for word in words:
                        if word in index2word_set:
                            n_words += 1
                            feature_vec = np.add(feature_vec, model[word])
                    if (n_words > 0):
                        feature_vec = np.divide(feature_vec, n_words)
                    return(feature_vec)

                def word_vectorizer(X):
                    """Where X is either X_test or X_train"""
                    data_set = pd.DataFrame(np.zeros([len(X),n_features]))
                    #vectors = np.zeros(n_features)
                    #dividend = 0 # you can remove this one right?


                    for i in range(len(X)):
                        sent_list = []
                        strings = X.iloc[i].split()
                        for j in range(len(strings)):
                            word = prep2(strings[j].lower())
                            if len(word) > 0:
                                clean_word = re.sub(r'[^\w\s]','',word[0])
                                sent_list.append(clean_word)
                                sent = " ".join(sent_list)

                                data_set.iloc[i] = avg_feature_vector(sent, model=word2vec_model, num_features=n_features, index2word_set=index2word_set)

                    return(data_set)

                w2v_train_df_new = word_vectorizer(X_train)
                w2v_test_df_new = word_vectorizer(X_test)

                clf_lg = LogisticRegression().fit(w2v_train_df_new, y_train)
                pred_proba_log = clf_lg.predict_proba(w2v_test_df_new)[:,1]

                auc = round(metrics.roc_auc_score(y_test, pred_proba_log), 3)

                
                
                #------------------------------------------------------------------------------------------------------

                
                
                model_name = "model_" + str(counter)
                hyper_parameters[model_name] = (auc,[fs, wc, mwc, s])
                                
                
                now = time.time()
                time_spend =  now - program_starts
                time_model = time_spend/counter
                total_time = (time_model) * total_number_of_models 
                
                print(str(counter)+"/"+str(total_number_of_models)+" done" + "\tAUC: {}\t hyper_par: {}, {}, {}, {}\t timer: {:.3} minutes".format(auc, fs, wc, mwc, s, time_spend/60))
                print("\tExpected time pr. model: {:.3} minutes".format(time_model/60))               
                print("\tExpected time total: {:.3} minutes".format(total_time/60))
                print("\tExpected time remaining: {:.3} minutes\n".format((total_time-time_spend)/60))

                counter += 1

1/180 done	AUC: 0.906	 hyper_par: 300, 4, 15, 0.0001	 timer: 3.42 minutes
	Expected time pr. model: 3.42 minutes
	Expected time total: 6.16e+02 minutes
	Expected time remaining: 6.13e+02 minutes

2/180 done	AUC: 0.904	 hyper_par: 300, 4, 15, 5e-05	 timer: 6.49 minutes
	Expected time pr. model: 3.24 minutes
	Expected time total: 5.84e+02 minutes
	Expected time remaining: 5.77e+02 minutes

3/180 done	AUC: 0.903	 hyper_par: 300, 4, 20, 0.0001	 timer: 9.89 minutes
	Expected time pr. model: 3.3 minutes
	Expected time total: 5.93e+02 minutes
	Expected time remaining: 5.84e+02 minutes

4/180 done	AUC: 0.903	 hyper_par: 300, 4, 20, 5e-05	 timer: 12.7 minutes
	Expected time pr. model: 3.17 minutes
	Expected time total: 5.71e+02 minutes
	Expected time remaining: 5.59e+02 minutes

5/180 done	AUC: 0.907	 hyper_par: 300, 4, 25, 0.0001	 timer: 15.9 minutes
	Expected time pr. model: 3.17 minutes
	Expected time total: 5.71e+02 minutes
	Expected time remaining: 5.55e+02 minutes

6/180 done	AUC: 0.904	 

In [144]:
# Assesing the models:
good_list = []

for i, j in hyper_parameters.items():
    if j[0] >= 0.908:
        print(j)

(0.908, [350, 10, 15, 5e-05])
(0.908, [350, 12, 20, 0.0001])
(0.908, [350, 12, 25, 0.0001])
(0.909, [400, 8, 15, 0.0001])
(0.908, [400, 10, 15, 0.0001])
(0.908, [400, 12, 20, 0.0001])
(0.908, [450, 5, 15, 0.0001])
(0.908, [450, 10, 15, 0.0001])
(0.908, [450, 12, 20, 0.0001])
(0.909, [500, 4, 20, 0.0001])
(0.908, [550, 10, 20, 0.0001])


In [142]:
hyper_parameters2 = hyper_parameters.copy()

In [143]:
# A pickle for the word2vec implimentation
output = open('hyper_parameters2.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(hyper_parameters2, output)

output.close()

# Third run
investigating that (0.912, [300, 10, 20, 0.001])

In [145]:
# New elauated parameters
feature_size_grid = [300]    # Word vector dimensionality  
window_context_grid = [8,10,12]          # Context window size                                                                                    
min_word_count_grid = [18,20,22]   # Minimum word count             HERE           
sample_grid = [1e-3,5e-4]   # Downsample setting for frequent words


In [146]:
all_models = defaultdict()
counter = 0

for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                #print(counter)
                counter += 1
                
total_number_of_models = counter
total_number_of_models

18

In [149]:
#all_models = defaultdict()
counter = 1
hyper_parameters = defaultdict(list)


program_starts = time.time()
for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                                
                spec_model = gensim.models.Word2Vec(sentences, size=fs, window=wc, min_count=mwc, sample=s, sg = 1, seed = 42, workers = 8) # helps, but still not deterministic
                
                #------------------------------------------------------------------------------------------------------
                
                word2vec_model = spec_model.wv
    
                n_features = word2vec_model.word_vec("potus").shape[0]
                index2word_set = set(word2vec_model.index2word)

                def avg_feature_vector(sentence, model, num_features, index2word_set):
                    words = sentence.split()
                    feature_vec = np.zeros((num_features, ), dtype='float32')
                    n_words = 0
                    for word in words:
                        if word in index2word_set:
                            n_words += 1
                            feature_vec = np.add(feature_vec, model[word])
                    if (n_words > 0):
                        feature_vec = np.divide(feature_vec, n_words)
                    return(feature_vec)

                def word_vectorizer(X):
                    """Where X is either X_test or X_train"""
                    data_set = pd.DataFrame(np.zeros([len(X),n_features]))
                    #vectors = np.zeros(n_features)
                    #dividend = 0 # you can remove this one right?


                    for i in range(len(X)):
                        sent_list = []
                        strings = X.iloc[i].split()
                        for j in range(len(strings)):
                            word = prep2(strings[j].lower())
                            if len(word) > 0:
                                clean_word = re.sub(r'[^\w\s]','',word[0])
                                sent_list.append(clean_word)
                                sent = " ".join(sent_list)

                                data_set.iloc[i] = avg_feature_vector(sent, model=word2vec_model, num_features=n_features, index2word_set=index2word_set)

                    return(data_set)

                w2v_train_df_new = word_vectorizer(X_train)
                w2v_test_df_new = word_vectorizer(X_test)

                clf_lg = LogisticRegression().fit(w2v_train_df_new, y_train)
                pred_proba_log = clf_lg.predict_proba(w2v_test_df_new)[:,1]

                auc = round(metrics.roc_auc_score(y_test, pred_proba_log), 3)

                
                
                #------------------------------------------------------------------------------------------------------

                
                
                model_name = "model_" + str(counter)
                hyper_parameters[model_name] = (auc,[fs, wc, mwc, s])
                                
                
                now = time.time()
                time_spend =  now - program_starts
                time_model = time_spend/counter
                total_time = (time_model) * total_number_of_models 
                
                print(str(counter)+"/"+str(total_number_of_models)+" done" + "\tAUC: {}\t hyper_par: {}, {}, {}, {}\t timer: {:.3} minutes".format(auc, fs, wc, mwc, s, time_spend/60))
                print("\tExpected time pr. model: {:.3} minutes".format(time_model/60))               
                print("\tExpected time total: {:.3} minutes".format(total_time/60))
                print("\tExpected time remaining: {:.3} minutes\n".format((total_time-time_spend)/60))

                counter += 1

1/18 done	AUC: 0.898	 hyper_par: 300, 8, 18, 0.001	 timer: 5.05 minutes
	Expected time pr. model: 5.05 minutes
	Expected time total: 90.9 minutes
	Expected time remaining: 85.8 minutes

2/18 done	AUC: 0.901	 hyper_par: 300, 8, 18, 0.0005	 timer: 9.64 minutes
	Expected time pr. model: 4.82 minutes
	Expected time total: 86.7 minutes
	Expected time remaining: 77.1 minutes

3/18 done	AUC: 0.898	 hyper_par: 300, 8, 20, 0.001	 timer: 14.6 minutes
	Expected time pr. model: 4.88 minutes
	Expected time total: 87.8 minutes
	Expected time remaining: 73.1 minutes

4/18 done	AUC: 0.903	 hyper_par: 300, 8, 20, 0.0005	 timer: 19.7 minutes
	Expected time pr. model: 4.92 minutes
	Expected time total: 88.5 minutes
	Expected time remaining: 68.8 minutes

5/18 done	AUC: 0.9	 hyper_par: 300, 8, 22, 0.001	 timer: 24.6 minutes
	Expected time pr. model: 4.92 minutes
	Expected time total: 88.5 minutes
	Expected time remaining: 63.9 minutes

6/18 done	AUC: 0.902	 hyper_par: 300, 8, 22, 0.0005	 timer: 29.2 minut

In [ ]:
# indeed the high score at 300, 10, 20, 0.001 the first time was luck. This is not a giving neighborhood

In [151]:
# Assesing the models:
good_list = []

for i, j in hyper_parameters.items():
    if j[0] >= 0.902:
        print(j)

(0.903, [300, 8, 20, 0.0005])
(0.902, [300, 8, 22, 0.0005])
(0.902, [300, 12, 18, 0.001])
(0.902, [300, 12, 22, 0.001])
(0.907, [300, 12, 22, 0.0005])


In [152]:
hyper_parameters3 = hyper_parameters.copy()

In [153]:
# A pickle for the word2vec implimentation
output = open('hyper_parameters3.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(hyper_parameters3, output)

output.close()

# Fouth run
investigating that (0.91, [450, 10, 30, 0.0001])

Which seems more pormising since you fund these in the seacond rund:

- (0.908, [450, 5, 15, 0.0001])
- (0.908, [450, 10, 15, 0.0001])
- (0.908, [450, 12, 20, 0.0001])

In [154]:
# New elauated parameters
feature_size_grid = [450]    # Word vector dimensionality  
window_context_grid = [5,8,10,12]          # Context window size                                                                                    
min_word_count_grid = [15,20,25,30,35]   # Minimum word count             HERE           
sample_grid = [1e-4]   # Downsample setting for frequent words


In [155]:
all_models = defaultdict()
counter = 0

for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                #print(counter)
                counter += 1
                
total_number_of_models = counter
total_number_of_models

20

In [156]:
#all_models = defaultdict()
counter = 1
hyper_parameters = defaultdict(list)


program_starts = time.time()
for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                                
                spec_model = gensim.models.Word2Vec(sentences, size=fs, window=wc, min_count=mwc, sample=s, sg = 1, seed = 42, workers = 8) # helps, but still not deterministic
                
                #------------------------------------------------------------------------------------------------------
                
                word2vec_model = spec_model.wv
    
                n_features = word2vec_model.word_vec("potus").shape[0]
                index2word_set = set(word2vec_model.index2word)

                def avg_feature_vector(sentence, model, num_features, index2word_set):
                    words = sentence.split()
                    feature_vec = np.zeros((num_features, ), dtype='float32')
                    n_words = 0
                    for word in words:
                        if word in index2word_set:
                            n_words += 1
                            feature_vec = np.add(feature_vec, model[word])
                    if (n_words > 0):
                        feature_vec = np.divide(feature_vec, n_words)
                    return(feature_vec)

                def word_vectorizer(X):
                    """Where X is either X_test or X_train"""
                    data_set = pd.DataFrame(np.zeros([len(X),n_features]))
                    #vectors = np.zeros(n_features)
                    #dividend = 0 # you can remove this one right?


                    for i in range(len(X)):
                        sent_list = []
                        strings = X.iloc[i].split()
                        for j in range(len(strings)):
                            word = prep2(strings[j].lower())
                            if len(word) > 0:
                                clean_word = re.sub(r'[^\w\s]','',word[0])
                                sent_list.append(clean_word)
                                sent = " ".join(sent_list)

                                data_set.iloc[i] = avg_feature_vector(sent, model=word2vec_model, num_features=n_features, index2word_set=index2word_set)

                    return(data_set)

                w2v_train_df_new = word_vectorizer(X_train)
                w2v_test_df_new = word_vectorizer(X_test)

                clf_lg = LogisticRegression().fit(w2v_train_df_new, y_train)
                pred_proba_log = clf_lg.predict_proba(w2v_test_df_new)[:,1]

                auc = round(metrics.roc_auc_score(y_test, pred_proba_log), 3)

                
                
                #------------------------------------------------------------------------------------------------------

                
                
                model_name = "model_" + str(counter)
                hyper_parameters[model_name] = (auc,[fs, wc, mwc, s])
                                
                
                now = time.time()
                time_spend =  now - program_starts
                time_model = time_spend/counter
                total_time = (time_model) * total_number_of_models 
                
                print(str(counter)+"/"+str(total_number_of_models)+" done" + "\tAUC: {}\t hyper_par: {}, {}, {}, {}\t timer: {:.3} minutes".format(auc, fs, wc, mwc, s, time_spend/60))
                print("\tExpected time pr. model: {:.3} minutes".format(time_model/60))               
                print("\tExpected time total: {:.3} minutes".format(total_time/60))
                print("\tExpected time remaining: {:.3} minutes\n".format((total_time-time_spend)/60))

                counter += 1

1/20 done	AUC: 0.903	 hyper_par: 450, 5, 15, 0.0001	 timer: 3.94 minutes
	Expected time pr. model: 3.94 minutes
	Expected time total: 78.9 minutes
	Expected time remaining: 74.9 minutes

2/20 done	AUC: 0.904	 hyper_par: 450, 5, 20, 0.0001	 timer: 7.66 minutes
	Expected time pr. model: 3.83 minutes
	Expected time total: 76.6 minutes
	Expected time remaining: 68.9 minutes

3/20 done	AUC: 0.906	 hyper_par: 450, 5, 25, 0.0001	 timer: 11.2 minutes
	Expected time pr. model: 3.73 minutes
	Expected time total: 74.6 minutes
	Expected time remaining: 63.5 minutes

4/20 done	AUC: 0.904	 hyper_par: 450, 5, 30, 0.0001	 timer: 14.7 minutes
	Expected time pr. model: 3.67 minutes
	Expected time total: 73.4 minutes
	Expected time remaining: 58.7 minutes

5/20 done	AUC: 0.903	 hyper_par: 450, 5, 35, 0.0001	 timer: 18.2 minutes
	Expected time pr. model: 3.65 minutes
	Expected time total: 72.9 minutes
	Expected time remaining: 54.7 minutes

6/20 done	AUC: 0.91	 hyper_par: 450, 8, 15, 0.0001	 timer: 23.3 m

In [158]:
# Assesing the models:
good_list = []

for i, j in hyper_parameters.items():
    if j[0] >= 0.908:
        print(j)

(0.91, [450, 8, 15, 0.0001])
(0.911, [450, 10, 35, 0.0001])


In [159]:
hyper_parameters4 = hyper_parameters.copy()

In [160]:
# A pickle for the word2vec implimentation
output = open('hyper_parameters4.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(hyper_parameters4, output)

output.close()

# Fifth (and last?) run: 

In [161]:
# New elauated parameters
feature_size_grid = [450]    # Word vector dimensionality  
window_context_grid = [8,10]          # Context window size                                                                                    
min_word_count_grid = [15,35,40,45]   # Minimum word count             HERE           
sample_grid = [1e-4]   # Downsample setting for frequent words


In [162]:
all_models = defaultdict()
counter = 0

for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                #print(counter)
                counter += 1
                
total_number_of_models = counter
total_number_of_models

8

In [163]:
#all_models = defaultdict()
counter = 1
hyper_parameters = defaultdict(list)


program_starts = time.time()
for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                                
                spec_model = gensim.models.Word2Vec(sentences, size=fs, window=wc, min_count=mwc, sample=s, sg = 1, seed = 42, workers = 8) # helps, but still not deterministic
                
                #------------------------------------------------------------------------------------------------------
                
                word2vec_model = spec_model.wv
    
                n_features = word2vec_model.word_vec("potus").shape[0]
                index2word_set = set(word2vec_model.index2word)

                def avg_feature_vector(sentence, model, num_features, index2word_set):
                    words = sentence.split()
                    feature_vec = np.zeros((num_features, ), dtype='float32')
                    n_words = 0
                    for word in words:
                        if word in index2word_set:
                            n_words += 1
                            feature_vec = np.add(feature_vec, model[word])
                    if (n_words > 0):
                        feature_vec = np.divide(feature_vec, n_words)
                    return(feature_vec)

                def word_vectorizer(X):
                    """Where X is either X_test or X_train"""
                    data_set = pd.DataFrame(np.zeros([len(X),n_features]))
                    #vectors = np.zeros(n_features)
                    #dividend = 0 # you can remove this one right?


                    for i in range(len(X)):
                        sent_list = []
                        strings = X.iloc[i].split()
                        for j in range(len(strings)):
                            word = prep2(strings[j].lower())
                            if len(word) > 0:
                                clean_word = re.sub(r'[^\w\s]','',word[0])
                                sent_list.append(clean_word)
                                sent = " ".join(sent_list)

                                data_set.iloc[i] = avg_feature_vector(sent, model=word2vec_model, num_features=n_features, index2word_set=index2word_set)

                    return(data_set)

                w2v_train_df_new = word_vectorizer(X_train)
                w2v_test_df_new = word_vectorizer(X_test)

                clf_lg = LogisticRegression().fit(w2v_train_df_new, y_train)
                pred_proba_log = clf_lg.predict_proba(w2v_test_df_new)[:,1]

                auc = round(metrics.roc_auc_score(y_test, pred_proba_log), 3)

                
                
                #------------------------------------------------------------------------------------------------------

                
                
                model_name = "model_" + str(counter)
                hyper_parameters[model_name] = (auc,[fs, wc, mwc, s])
                                
                
                now = time.time()
                time_spend =  now - program_starts
                time_model = time_spend/counter
                total_time = (time_model) * total_number_of_models 
                
                print(str(counter)+"/"+str(total_number_of_models)+" done" + "\tAUC: {}\t hyper_par: {}, {}, {}, {}\t timer: {:.3} minutes".format(auc, fs, wc, mwc, s, time_spend/60))
                print("\tExpected time pr. model: {:.3} minutes".format(time_model/60))               
                print("\tExpected time total: {:.3} minutes".format(total_time/60))
                print("\tExpected time remaining: {:.3} minutes\n".format((total_time-time_spend)/60))

                counter += 1

1/8 done	AUC: 0.902	 hyper_par: 450, 8, 15, 0.0001	 timer: 5.02 minutes
	Expected time pr. model: 5.02 minutes
	Expected time total: 40.2 minutes
	Expected time remaining: 35.1 minutes

2/8 done	AUC: 0.908	 hyper_par: 450, 8, 35, 0.0001	 timer: 9.64 minutes
	Expected time pr. model: 4.82 minutes
	Expected time total: 38.6 minutes
	Expected time remaining: 28.9 minutes

3/8 done	AUC: 0.898	 hyper_par: 450, 8, 40, 0.0001	 timer: 14.2 minutes
	Expected time pr. model: 4.73 minutes
	Expected time total: 37.9 minutes
	Expected time remaining: 23.7 minutes

4/8 done	AUC: 0.901	 hyper_par: 450, 8, 45, 0.0001	 timer: 18.6 minutes
	Expected time pr. model: 4.65 minutes
	Expected time total: 37.2 minutes
	Expected time remaining: 18.6 minutes

5/8 done	AUC: 0.902	 hyper_par: 450, 10, 15, 0.0001	 timer: 24.2 minutes
	Expected time pr. model: 4.84 minutes
	Expected time total: 38.7 minutes
	Expected time remaining: 14.5 minutes

6/8 done	AUC: 0.905	 hyper_par: 450, 10, 35, 0.0001	 timer: 29.3 minu

In [164]:
# Assesing the models:
good_list = []

for i, j in hyper_parameters.items():
    if j[0] >= 0.908:
        print(j)

(0.908, [450, 8, 35, 0.0001])
(0.908, [450, 10, 40, 0.0001])


In [165]:
hyper_parameters5 = hyper_parameters.copy()

In [166]:
# A pickle for the word2vec implimentation
output = open('hyper_parameters5.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(hyper_parameters5, output)

output.close()

# Sith run..

In [167]:
# New elauated parameters
feature_size_grid = [450]    # Word vector dimensionality  
window_context_grid = [10]          # Context window size                                                                                    
min_word_count_grid = [30,35,40]   # Minimum word count             HERE           
sample_grid = [1e-4]   # Downsample setting for frequent words


In [168]:
all_models = defaultdict()
counter = 0

for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                #print(counter)
                counter += 1
                
total_number_of_models = counter
total_number_of_models

3

In [169]:
#all_models = defaultdict()
counter = 1
hyper_parameters = defaultdict(list)


program_starts = time.time()
for fs in feature_size_grid:
    for wc in window_context_grid:
        for mwc in min_word_count_grid:
            for s in sample_grid:
                                
                spec_model = gensim.models.Word2Vec(sentences, size=fs, window=wc, min_count=mwc, sample=s, sg = 1, seed = 42, workers = 8) # helps, but still not deterministic
                
                #------------------------------------------------------------------------------------------------------
                
                word2vec_model = spec_model.wv
    
                n_features = word2vec_model.word_vec("potus").shape[0]
                index2word_set = set(word2vec_model.index2word)

                def avg_feature_vector(sentence, model, num_features, index2word_set):
                    words = sentence.split()
                    feature_vec = np.zeros((num_features, ), dtype='float32')
                    n_words = 0
                    for word in words:
                        if word in index2word_set:
                            n_words += 1
                            feature_vec = np.add(feature_vec, model[word])
                    if (n_words > 0):
                        feature_vec = np.divide(feature_vec, n_words)
                    return(feature_vec)

                def word_vectorizer(X):
                    """Where X is either X_test or X_train"""
                    data_set = pd.DataFrame(np.zeros([len(X),n_features]))
                    #vectors = np.zeros(n_features)
                    #dividend = 0 # you can remove this one right?


                    for i in range(len(X)):
                        sent_list = []
                        strings = X.iloc[i].split()
                        for j in range(len(strings)):
                            word = prep2(strings[j].lower())
                            if len(word) > 0:
                                clean_word = re.sub(r'[^\w\s]','',word[0])
                                sent_list.append(clean_word)
                                sent = " ".join(sent_list)

                                data_set.iloc[i] = avg_feature_vector(sent, model=word2vec_model, num_features=n_features, index2word_set=index2word_set)

                    return(data_set)

                w2v_train_df_new = word_vectorizer(X_train)
                w2v_test_df_new = word_vectorizer(X_test)

                clf_lg = LogisticRegression().fit(w2v_train_df_new, y_train)
                pred_proba_log = clf_lg.predict_proba(w2v_test_df_new)[:,1]

                auc = round(metrics.roc_auc_score(y_test, pred_proba_log), 3)

                
                
                #------------------------------------------------------------------------------------------------------

                
                
                model_name = "model_" + str(counter)
                hyper_parameters[model_name] = (auc,[fs, wc, mwc, s])
                                
                
                now = time.time()
                time_spend =  now - program_starts
                time_model = time_spend/counter
                total_time = (time_model) * total_number_of_models 
                
                print(str(counter)+"/"+str(total_number_of_models)+" done" + "\tAUC: {}\t hyper_par: {}, {}, {}, {}\t timer: {:.3} minutes".format(auc, fs, wc, mwc, s, time_spend/60))
                print("\tExpected time pr. model: {:.3} minutes".format(time_model/60))               
                print("\tExpected time total: {:.3} minutes".format(total_time/60))
                print("\tExpected time remaining: {:.3} minutes\n".format((total_time-time_spend)/60))

                counter += 1

1/3 done	AUC: 0.901	 hyper_par: 450, 10, 30, 0.0001	 timer: 5.39 minutes
	Expected time pr. model: 5.39 minutes
	Expected time total: 16.2 minutes
	Expected time remaining: 10.8 minutes

2/3 done	AUC: 0.902	 hyper_par: 450, 10, 35, 0.0001	 timer: 10.6 minutes
	Expected time pr. model: 5.28 minutes
	Expected time total: 15.9 minutes
	Expected time remaining: 5.28 minutes

3/3 done	AUC: 0.905	 hyper_par: 450, 10, 40, 0.0001	 timer: 15.6 minutes
	Expected time pr. model: 5.2 minutes
	Expected time total: 15.6 minutes
	Expected time remaining: 0.0 minutes



In [170]:
# Assesing the models:
good_list = []

for i, j in hyper_parameters.items():
    if j[0] >= 0.908:
        print(j)

In [171]:
hyper_parameters6 = hyper_parameters.copy()

In [172]:
# A pickle for the word2vec implimentation
output = open('hyper_parameters6.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(hyper_parameters6, output)

output.close()

# I Choose to use the hyperparameters 450, 10, 40, 0.0001 for the model